# Nature Conservancy Fish Classification - Bounding Box Crops

### Imports & Environment

In [1]:
import os
import ujson as json
import PIL
import random
import matplotlib.pyplot as plt

from glob import glob
from collections import defaultdict

ROOT_DIR = os.getcwd()
DATA_HOME_DIR = ROOT_DIR + '/data'
%matplotlib inline

In [2]:
# paths
data_path = DATA_HOME_DIR + '/' 
full_train_path = data_path + 'train_full/'
crop_path = data_path + 'cropped/'

# data
classes = ["ALB", "BET", "DOL", "LAG", "OTHER", "SHARK", "YFT"]
nb_classes = len(classes)

### Cropping Images to Bounding Box Coordinates

So, because a few fish have been relabeled in the training set, the classes won't line up exactly with the classes in the annotation files. 

As a roundabout way of getting around this, I create a dictionary mapping image files with their respective classes so that when I iterate through the annotation files I can pair them up on the fly. 

In [3]:
class_dict = defaultdict(str)

for fp in glob(full_train_path + '*/*g'):
    cls = fp.split('/')[-2]
    im = fp.split('/')[-1]
    class_dict[im] = cls
    
print("Image Records:", len(class_dict.keys()))

Image Records: 3777


Load bounding box data from json

In [4]:
anno_classes = ['alb', 'bet', 'dol', 'lag', 'other', 'shark', 'yft']
bb_json = {}

for c in anno_classes:
    j = json.load(open('bb_annotations/{}.json'.format(c), 'r'))
    for l in j:
        if 'annotations' in l.keys() and len(l['annotations'])>0:
            bb_json[l['filename'].split('/')[-1]] = sorted(
                l['annotations'], key=lambda x: x['height']*x['width'])[-1]

Helper function for converting coordinates to resized image

In [5]:
bb_params = ['height', 'width', 'x', 'y']
def convert_bb(bb):
    cropsize = 224
    size = bb["size"]
    bb = [bb[p] for p in bb_params]
    
    # conversion factors
    conv_x = (640. / size[0])
    conv_y = (360. / size[1])
    
    # make the size conversions
    width, height = size[0]*conv_x, size[1]*conv_y
    x = bb[2]*conv_x
    y = bb[3]*conv_y
    
    # offset/padding adjustments
    x = max(x - 10, 0)
    y = max(y - 10, 0)
    
    if x + cropsize > width:
        x = width - cropsize
    if y + cropsize > height:
        y = height - cropsize
    
    bb[0] = cropsize
    bb[1] = cropsize
    bb[2] = x
    bb[3] = y
    return bb

Then we iterate through each class, grab the annotations for that class, crop the image down to a square around the fish, and save it to my cropped data directory. 

In [6]:
j = json.load(open('bb_annotations/{}.json'.format(c.lower()), 'r'))

for c in classes:
    fns = glob(full_train_path + c + '/*.jpg')
    
    for fn in fns:
        f_id = fn.split("/")[-1]
        cls = class_dict[f_id]
        im = PIL.Image.open('{0}{1}/{2}'.format(full_train_path, cls, f_id))
        width, height = im.size
    
        if not f_id in bb_json.keys():
            continue
#             x = random.uniform(0, width)
#             y = random.uniform(0, height)
#             bb = {"height": 0, "width": 0, "x": x, "y": y, "size": im.size}
#             bb = convert_bb(bb)
#             im = im.resize((640, 360))
#             cropped = im.crop((bb[2], bb[3], bb[2] + bb[1], bb[3] + bb[0]))
#             cropped.save(fn.replace(full_train_path, crop_path)) 
        else:
            anno = bb_json[f_id]
            x, y = anno["x"], anno["y"]
            bb = {"height": 0, "width": 0, "x": x, "y": y, "size": im.size}
            bb = convert_bb(bb)
            im = im.resize((640, 360))
            cropped = im.crop((bb[2], bb[3], bb[2] + bb[1], bb[3] + bb[0]))
            cropped.save(fn.replace(full_train_path, crop_path)) 

In [7]:
crops = glob(crop_path + '*/*g')

print("Cropped Image Records:", len(crops))

Cropped Image Records: 3297


Looks like a few records were lost in the process (in addition to the ones printed out above), but after looking through some of the missing annotations, there's generally a good reason, such as multiple fish or obscured fish. 